<a href="https://colab.research.google.com/github/NguyenNhan1992/Advanced-Deep-Learning-with-Keras/blob/master/CVAE_3D_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q keras

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import keras
from keras.layers import Dense, Input
from keras.layers import Conv2D, Flatten, Lambda, Conv3D
from keras.layers import Reshape, Conv2DTranspose, Conv3DTranspose
from keras.models import Model
from keras.datasets import mnist
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K
from keras.utils import to_categorical

import numpy as np
import matplotlib.pyplot as plt
import argparse
import os

Using TensorFlow backend.


In [0]:
def sampling(args):
    """Implements reparameterization trick by sampling
    from a gaussian with zero mean and std=1.
    Arguments:
        args (tensor): mean and log of variance of Q(z|X)
    Returns:
        sampled latent vector (tensor)
    """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [0]:
image_size = 28
num_labels = 10
input_shape = (8, image_size, image_size, 1)
label_shape = (num_labels, )
batch_size = 128
kernel_size = 3
filters = 16
latent_dim = 2

In [5]:
inputs = Input(shape=input_shape, name='encoder_input')
x = Conv3D(filters=16, kernel_size=(3,3,3), activation='relu', strides=2, padding='same')(inputs)
x = Conv3D(filters=32, kernel_size=(3,3,3), activation='relu', strides=2, padding='same')(x)
shape = K.int_shape(x)
# print(shape)
x = Flatten()(x)
x = Dense(16, activation='relu')(x)
z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)

# use reparameterization trick to push the sampling out as input
# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

# instantiate encoder model
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()

latent_inputs = Input(shape=(latent_dim,), name='z_sampling')

x = Dense(shape[1]*shape[2]*shape[3]*shape[4], activation='relu')(latent_inputs)
x = Reshape((shape[1], shape[2], shape[3], shape[4]))(x)

x = Conv3DTranspose(filters=32, kernel_size=(4,4,4), activation='relu', strides=2, padding='same')(x)
x = Conv3DTranspose(filters=16, kernel_size=(2,2,2), activation='relu', strides=2, padding='same')(x)

outputs = Conv3DTranspose(filters=1, kernel_size=(4,4,4), activation='sigmoid', padding='same', name='decoder_output')(x)

# instantiate decoder model
decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 8, 28, 28, 1) 0                                            
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 4, 14, 14, 16 448         encoder_input[0][0]              
__________________________________________________________________________________________________
conv3d_2 (Conv3D)               (None, 2, 7, 7, 32)  13856       conv3d_1[0][0]                   
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 3136)         0           conv3d_2[0][0]                   
__________________________________________________________________________________________________
dense_1 (D

In [0]:
outputs = decoder(encoder(inputs)[2])
cvae_3d = Model(inputs, outputs, name='cvae')

In [0]:
models = (encoder, decoder)

In [0]:
reconstruction_loss = binary_crossentropy(K.flatten(inputs),  K.flatten(outputs))

In [10]:
beta = 1.0
reconstruction_loss *= 28 * 28
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5 * beta
cvae_loss = K.mean(reconstruction_loss + kl_loss)
cvae_3d.add_loss(cvae_loss)
cvae_3d.compile(optimizer='rmsprop')
cvae_3d.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   (None, 8, 28, 28, 1)      0         
_________________________________________________________________
encoder (Model)              [(None, 2), (None, 2), (N 64564     
_________________________________________________________________
decoder (Model)              (None, 8, 28, 28, 1)      80113     
Total params: 144,677
Trainable params: 144,677
Non-trainable params: 0
_________________________________________________________________


In [11]:
X_train = np.random.rand(1200, 8,28,28)
x_test = np.random.rand(10,8, 28,28)
X_train = np.expand_dims(X_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)
print(X_train.shape)

(1200, 8, 28, 28, 1)


In [12]:
cvae_3d.fit(X_train, epochs=10,
                 batch_size=12,
                 validation_data=(x_test, None))

Train on 1200 samples, validate on 10 samples
Epoch 1/10
1200/1200 [==============================] - 4s 3ms/step - loss: 543.4330 - val_loss: 543.4299
Epoch 2/10
1200/1200 [==============================] - 2s 1ms/step - loss: 543.4279 - val_loss: 543.4282
Epoch 3/10
1200/1200 [==============================] - 2s 1ms/step - loss: 543.4270 - val_loss: 543.4294
Epoch 4/10
1200/1200 [==============================] - 2s 1ms/step - loss: 543.4260 - val_loss: 543.4313
Epoch 5/10
1200/1200 [==============================] - 2s 1ms/step - loss: 543.4229 - val_loss: 543.4251
Epoch 6/10
1200/1200 [==============================] - 2s 1ms/step - loss: 543.4195 - val_loss: 543.4274
Epoch 7/10
1200/1200 [==============================] - 2s 1ms/step - loss: 543.4128 - val_loss: 543.4301
Epoch 8/10
1200/1200 [==============================] - 2s 1ms/step - loss: 543.4069 - val_loss: 543.4399
Epoch 9/10
1200/1200 [==============================] - 2s 1ms/step - loss: 543.4027 - val_loss: 543.4351
